# Biochar and charcrete database

## 1. Set the project and import Charcrete database

In [1]:
# === Switch to kernel bw25! ===

# Brightway libraries
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw_processing as bwp

# General libraries
import numpy as np
import pandas as pd
from fs.zipfs import ZipFS
import json                        # Library for working with json files
from pathlib import Path           # Library for working with paths in different OS     
import matplotlib.pyplot as plt    # Library for creating plots

In [3]:
bd.projects.set_current("ei38-teaching-25")

In [ ]:
if "Charcrete_38" in bd.databases:
    print("Charcrete_38 database already exists")
else:
    # 1. Specify filepath to your foreground inventories.
    charcrete_path = "/data/Charcrete_database_38.xlsx"
    # 2. Create an instance of a class that contains basic methods for importing a database from an excel file.
    charcrete = bi.ExcelImporter(charcrete_path)  
    # 3. `apply_strategies` is one of such basic methods, it makes sure units, locations, etc are in correct format.
    charcrete.apply_strategies()
    # 4. Next step is to link your foreground exchanges to existing databases by matching relevant exchanges fields.
    charcrete.match_database("biosphere3", fields=("name", "unit", "categories"))
    charcrete.match_database("ecoinvent 3.8 cutoff", fields=("name", "location", "unit"))
    charcrete.metadata.pop(None)  # Remove metadata None entry. TODO
    # 5. If everything is linked, write database so that it is saved in your project.
    if charcrete.all_linked:
        charcrete.write_database()

In [4]:
bd.databases

Databases dictionary with 4 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	biosphere3
	ei 3.8 cutoff